In [1]:
import tensorflow as tf
import numpy as np

**`tf.clip_by_average_norm`**
**`tf.clip_by_global_norm`**
**`tf.clip_by_norm`**
**`tf.clip_by_value(t, clip_value_min, clip_value_max, name=None)`**
+ Clips tensor values to a specified min and max.

In [8]:
v = tf.constant([[1.0, 2.0, 4.0],[4.0, 5.0, 6.0]])
result = tf.clip_by_value(v, 2.5, 4.5)

with tf.Session() as sess:
    print(sess.run(v))
    print(sess.run(result))

[[1. 2. 4.]
 [4. 5. 6.]]
[[2.5 2.5 4. ]
 [4.  4.5 4.5]]


**`tf.train`**

+ **`tf.train.Optimizer()`**

In [ ]:
Class Optimizer 的基类，这个类定义了在训练模型时候添加一个操作的API
基本不会直接使用这个类，但是会用到它的子类比如 `tf.trian.GradientDescentOptimizer, AdagradOptimizer, MomentumOptimizer`

+ Base class for optimizers. This class defines the API to add Ops to train a model.
+ You never use this class directly, but instead instantiate one of its subclasses such as `GradientDescentOptimizer`, `AdagradOptimizer`, or `MomentumOptimizer`.

+ `Usage`

```python
# Create an optimizer with the desired parameters.
opt = GradientDescentOptimizer(learning_rate=0.1)
# Add Ops to the graph to minimize a cost by updating a list of variables.
# "cost" is a Tensor, and the list of variables contains tf.Variable
# objects.
opt_op = opt.minimize(cost, var_list=<list of variables>)
```

In [ ]:
tf.train.Optimizer
https://www.tensorflow.org/api_docs/python/tf/train/Optimizer

+ `Method`

```python
apply_gradients(
    grads_and_vars,
    global_step=None,
    name=None
)


compute_gradients(
    loss,
    var_list=None,
    gate_gradients=GATE_OP,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)

minimize(
    loss,
    global_step=None,
    var_list=None,
    gate_gradients=GATE_OP,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    name=None,
    grad_loss=None
)

```

```python
apply_gradients(
    grads_and_vars,
    global_step=None,
    name=None
)
```

Apply gradients to variables.

This is the second part of `minimize()`. It returns an `Operation` that applies gradients.


+ Args:
    - `grads_and_vars`: List of (gradient, variable) pairs as returned by `compute_gradients()`.
    - `global_step`: Optional `Variable` to increment by one after the variables have been updated.
    - `name`: Optional name for the returned operation. Default to the name passed to the `Optimizer` constructor.

- Returns:
    - An `Operation` that applies the specified gradients. If `global_step` was not None, that operation also increments `global_step`.

```python
compute_gradients(
    loss,
    var_list=None,
    gate_gradients=GATE_OP,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)
```

Compute gradients of loss for the variables in var_list.

This is the first part of minimize(). It returns a list of (gradient, variable) pairs where "gradient" is the gradient for "variable". Note that "gradient" can be a Tensor, an IndexedSlices, or None if there is no gradient for the given variable.
Args:

    loss: A Tensor containing the value to minimize or a callable taking no arguments which returns the value to minimize. When eager execution is enabled it must be a callable.
    var_list: Optional list or tuple of tf.Variable to update to minimize loss. Defaults to the list of variables collected in the graph under the key GraphKeys.TRAINABLE_VARIABLES.
    gate_gradients: How to gate the computation of gradients. Can be GATE_NONE, GATE_OP, or GATE_GRAPH.
    aggregation_method: Specifies the method used to combine gradient terms. Valid values are defined in the class AggregationMethod.
    colocate_gradients_with_ops: If True, try colocating gradients with the corresponding op.
    grad_loss: Optional. A Tensor holding the gradient computed for loss.

Returns:

A list of (gradient, variable) pairs. Variable is always present, but gradient can be None.

```python
minimize(
    loss,
    global_step=None,
    var_list=None,
    gate_gradients=GATE_OP,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    name=None,
    grad_loss=None
)

```

Add operations to minimize loss by updating var_list.

This method simply combines calls compute_gradients() and apply_gradients(). If you want to process the gradient before applying them call compute_gradients() and apply_gradients() explicitly instead of using this function.
Args:

    loss: A Tensor containing the value to minimize.
    global_step: Optional Variable to increment by one after the variables have been updated.
    var_list: Optional list or tuple of Variable objects to update to minimize loss. Defaults to the list of variables collected in the graph under the key GraphKeys.TRAINABLE_VARIABLES.
    gate_gradients: How to gate the computation of gradients. Can be GATE_NONE, GATE_OP, or GATE_GRAPH.
    aggregation_method: Specifies the method used to combine gradient terms. Valid values are defined in the class AggregationMethod.
    colocate_gradients_with_ops: If True, try colocating gradients with the corresponding op.
    name: Optional name for the returned operation.
    grad_loss: Optional. A Tensor holding the gradient computed for loss.

Returns:

An Operation that updates the variables in var_list. If global_step was not None, that operation also increments global_step.

In [ ]:
https://blog.csdn.net/lenbow/article/details/52218551
该操作不仅可以优化更新训练的模型参数，也可以为`全局步骤 / global step`计数

In [ ]:
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)

In [ ]:
返回一个优化更新后的`var_list`
如果`global_step`非`None`，该操作还会为`global_step`做自增操作

**`梯度修剪`**
+ tf.train.Optimizer().apply_gradients()
+ tf.train.Optimizer().compute_gradients()

梯度修剪主要避免训练造成的 `梯度爆炸` & `梯度消失` 问题

In [ ]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)
在调用`minimize()`方法时，底层实际做了两件事：
    计算所有`trainable_variable`的`gradient`
    将`gradient`作用于`trainable_variable`
在调用`sess.run(train_op)`时，会对`trainable_variable`进行更新

In [ ]:
### Processing gradients before applying them.

Calling `minimize()` takes care of both computing the gradients and
applying them to the variables.  If you want to process the gradients
before applying them you can instead use the optimizer in three steps:

1.  Compute the gradients with `compute_gradients()`.
2.  Process the gradients as you wish.
3.  Apply the processed gradients with `apply_gradients()`.



In [ ]:

Example:

```python
# Create an optimizer.
opt = GradientDescentOptimizer(learning_rate=0.1)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss, <list of variables>)

# grads_and_vars is a list of tuples (gradient, variable).  Do whatever you
# need to the 'gradient' part, for example cap them, etc.
capped_grads_and_vars = [(MyCapper(gv[0]), gv[1]) for gv in grads_and_vars]

# Ask the optimizer to apply the capped gradients.
opt.apply_gradients(capped_grads_and_vars)
```

```python
compute_gradients(
    loss,
    var_list=None,
    gate_gradients=GATE_OP,
    aggregation_method=None,
    colocate_gradients_with_ops=False,
    grad_loss=None
)
```

In [ ]:
计算`loss`中可训练的`var_list`中的梯度
相当于`minimize()`的第一步，返回 `(gradient, variable)` 对的 list

Compute gradients of `loss` for the variables in `var_list`.

This is the first part of minimize(). It returns a list of (gradient, variable) pairs where "gradient" is the gradient for "variable". Note that "gradient" can be a Tensor, an IndexedSlices, or None if there is no gradient for the given variable.
Args:

    loss: A Tensor containing the value to minimize or a callable taking no arguments which returns the value to minimize. When eager execution is enabled it must be a callable.
    var_list: Optional list or tuple of tf.Variable to update to minimize loss. Defaults to the list of variables collected in the graph under the key GraphKeys.TRAINABLE_VARIABLES.
    gate_gradients: How to gate the computation of gradients. Can be GATE_NONE, GATE_OP, or GATE_GRAPH.
    aggregation_method: Specifies the method used to combine gradient terms. Valid values are defined in the class AggregationMethod.
    colocate_gradients_with_ops: If True, try colocating gradients with the corresponding op.
    grad_loss: Optional. A Tensor holding the gradient computed for loss.

Returns:

A list of (gradient, variable) pairs. Variable is always present, but gradient can be None.

```python
apply_gradients(
    grads_and_vars,
    global_step=None,
    name=None
)
```

将`gradient`作用于`variables`
`minimize()`的第二部分，返回一个执行梯度更新的`Operation`

Apply gradients to variables.

This is the second part of `minimize()`. It returns an `Operation` that applies gradients.


+ Args:
    - `grads_and_vars`: List of (gradient, variable) pairs as returned by `compute_gradients()`.
    - `global_step`: Optional `Variable` to increment by one after the variables have been updated.
    - `name`: Optional name for the returned operation. Default to the name passed to the `Optimizer` constructor.

- Returns:
    - An `Operation` that applies the specified gradients. If `global_step` was not None, that operation also increments `global_step`.

In [ ]:
`Examples`

```python
# Create an optimizer with the desired parameters.
opt = GradientDescentOptimizer(learning_rate=0.1)
# Add Ops to the graph to minimize a cost by updating a list of variables.
# "cost" is a Tensor, and the list of variables contains tf.Variable
# objects.
opt_op = opt.minimize(cost, var_list=<list of variables>)

```

```python
# Create an optimizer.
opt = GradientDescentOptimizer(learning_rate=0.1)

# Compute the gradients for a list of variables.
grads_and_vars = opt.compute_gradients(loss, <list of variables>)

# grads_and_vars is a list of tuples (gradient, variable).  Do whatever you
# need to the 'gradient' part, for example cap them, etc.
capped_grads_and_vars = [(MyCapper(gv[0]), gv[1]) for gv in grads_and_vars]

# Ask the optimizer to apply the capped gradients.
opt.apply_gradients(capped_grads_and_vars)
```

**`tf.train.AdamOptimizer`**
**`tf.train.ExponentialMovingAverage`**
**`tf.train.GradientDescentOptimizer`**
**`tf.train.MomentumOptimizer`**
**`tf.train.Optimizer`**
**`tf.train.RMSPropOptimizer`**
**`tf.train.XXX`**

In [ ]:
tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')

Construct a new Adam optimizer.

Initialization:

$$m_0 := 0  ext{(Initialize initial 1st moment vector)}$$
$$v_0 := 0  ext{(Initialize initial 2nd moment vector)}$$
$$t := 0    ext{(Initialize timestep)}$$

The update rule for `variable` with gradient `g` uses an optimization
described at the end of section2 of the paper:

$$t := t + 1$$
$$lr_t :=   ext{learning\_rate} * \sqrt{1 - beta_2^t} / (1 - beta_1^t)$$

$$m_t := beta_1 * m_{t-1} + (1 - beta_1) * g$$
$$v_t := beta_2 * v_{t-1} + (1 - beta_2) * g * g$$
$$variable := variable - lr_t * m_t / (\sqrt{v_t} + \epsilon)$$

The default value of 1e-8 for epsilon might not be a good default in
general. For example, when training an Inception network on ImageNet a
current good choice is 1.0 or 0.1. Note that since AdamOptimizer uses the
formulation just before Section 2.1 of the Kingma and Ba paper rather than
the formulation in Algorithm 1, the "epsilon" referred to here is "epsilon
hat" in the paper.

The sparse implementation of this algorithm (used when the gradient is an
IndexedSlices object, typically because of `tf.gather` or an embedding
lookup in the forward pass) does apply momentum to variable slices even if
they were not used in the forward pass (meaning they have a gradient equal
to zero). Momentum decay (beta1) is also applied to the entire momentum
accumulator. This means that the sparse behavior is equivalent to the dense
behavior (in contrast to some momentum implementations which ignore momentum
unless a variable slice was actually used).

Args:
  learning_rate: A Tensor or a floating point value.  The learning rate.
  beta1: A float value or a constant float tensor.
    The exponential decay rate for the 1st moment estimates.
  beta2: A float value or a constant float tensor.
    The exponential decay rate for the 2nd moment estimates.
  epsilon: A small constant for numerical stability. This epsilon is
    "epsilon hat" in the Kingma and Ba paper (in the formula just before
    Section 2.1), not the epsilon in Algorithm 1 of the paper.
  use_locking: If True use locks for update operations.
  name: Optional name for the operations created when applying gradients.
    Defaults to "Adam".

@compatibility(eager)
When eager execution is enabled, `learning_rate`, `beta1`, `beta2`, and
`epsilon` can each be a callable that takes no arguments and returns the
actual value to use. This can be useful for changing these values across
different invocations of optimizer functions.
@end_compatibility

In [ ]:
相比较于 SGD 算法而言
1. 不容易陷入局部最优点
2. 速度更快

learning_rate: A Tensor or a floating point value. The learning rate.
beta1: A float value or a constant float tensor. The exponential decay rate for the 1st moment estimates.
beta2: A float value or a constant float tensor. The exponential decay rate for the 2nd moment estimates.
epsilon: A small constant for numerical stability. This epsilon is "epsilon hat" in the Kingma and Ba paper (in the formula just before Section 2.1), not the epsilon in Algorithm 1 of the paper.
use_locking: If True use locks for update operations.
name: Optional name for the operations created when applying gradients. Defaults to "Adam".·

In [ ]:
# x = tf.placeholder(tf.float32)
x = tf.Variable(tf.truncated_normal([1]), name="x")
goal = tf.pow(x-3,2, name="goal")


In [ ]:
with tf.Session() as sess:
    x.initializer.run()
    print x.eval()
    print goal.eval()

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_step = optimizer.minimize(goal)

In [ ]:
def train():
    with tf.Session() as sess:
        x.initializer.run()
        for i in range(10):
            print "x: ", x.eval()
            train_step.run()
            print "goal: ",goal.eval()    
train()

In [ ]:
y = tf.Variable(tf.truncated_normal([1]))
max_goal = tf.sin(y)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
# train_step = optimizer.minimize(tf.negative(max_goal))
train_step = optimizer.minimize(-1 * max_goal

In [ ]:
with tf.Session() as sess:
    y.initializer.run()
    for i in range(10):
        print "y: ", y.eval()
        train_step.run()
        print "max_goal: ",max_goal.eval()

In [ ]:
# compute_gradients 返回的是：A list of (gradient, variable) pairs
gra_and_var = optimizer.compute_gradients(goal)
train_step = optimizer.apply_gradients(gra_and_var)
train()

In [ ]:
clip_by_global_norm:修正梯度值
    用于控制梯度爆炸的问题。梯度爆炸和梯度弥散的原因一样，都是因为链式法则求导的关系，导致梯度的指数级衰减。为了避免梯度爆炸，需要对梯度进行修剪。
In [9]:

gradients, vriables = zip(*optimizer.compute_gradients(goal))
gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
train_step = optimizer.apply_gradients(zip(gradients, vriables))
train()



In [ ]:

exponential_decay 加入学习率衰减：
In [10]:

# global_step 记录当前是第几个batch
global_step = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
    3.0, global_step, 3, 0.3, staircase=True)
optimizer2 = tf.train.GradientDescentOptimizer(learning_rate)
gradients, vriables = zip(*optimizer2.compute_gradients(goal))
gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
train_step = optimizer2.apply_gradients(zip(gradients, vriables), 
                                       global_step=global_step)
with tf.Session() as sess:
        global_step.initializer.run()
        x.initializer.run()
        for i in range(10):
            print "x: ", x.eval()
            train_step.run()
            print "goal: ",goal.eval()



# tf.gradient & tf.stop_gradient

tf.gradients(ys, xs, grad_ys=None, name='gradients', stop_gradients=None)

In [ ]:
Constructs symbolic derivatives of sum of `ys` w.r.t. x in `xs`.

`ys` and `xs` are each a `Tensor` or a list of tensors.  `grad_ys`
is a list of `Tensor`, holding the gradients received by the
`ys`. The list must be the same length as `ys`.

`gradients()` adds ops to the graph to output the derivatives of `ys` with
respect to `xs`.  It returns a list of `Tensor` of length `len(xs)` where
each tensor is the `sum(dy/dx)` for y in `ys`.

In [ ]:
`grad_ys` is a list of tensors of the same length as `ys` that holds
the initial gradients for each y in `ys`.  When `grad_ys` is None,
we fill in a tensor of '1's of the shape of y for each y in `ys`.  A
user can provide their own initial `grad_ys` to compute the
derivatives using a different initial gradient for each y (e.g., if
one wanted to weight the gradient differently for each value in
each y).

In [ ]:
`stop_gradients` is a `Tensor` or a list of tensors to be considered constant
with respect to all `xs`. These tensors will not be backpropagated through,
as though they had been explicitly disconnected using `stop_gradient`.  Among
other things, this allows computation of partial derivatives as opposed to
total derivatives. For example:

```python
a = tf.constant(0.)
b = 2 * a
g = tf.gradients(a + b, [a, b], stop_gradients=[a, b])
```

Here the partial derivatives `g` evaluate to `[1.0, 1.0]`, compared to the
total derivatives `tf.gradients(a + b, [a, b])`, which take into account the
influence of `a` on `b` and evaluate to `[3.0, 1.0]`.  Note that the above is
equivalent to:

```python
a = tf.stop_gradient(tf.constant(0.))
b = tf.stop_gradient(2 * a)
g = tf.gradients(a + b, [a, b])
```

`stop_gradients` provides a way of stopping gradient after the graph has
already been constructed, as compared to `tf.stop_gradient` which is used
during graph construction.  When the two approaches are combined,
backpropagation stops at both `tf.stop_gradient` nodes and nodes in
`stop_gradients`, whichever is encountered first.

All integer tensors are considered constant with respect to all `xs`, as if
they were included in `stop_gradients`.

In [ ]:
ys: A `Tensor` or list of tensors to be differentiated.
xs: A `Tensor` or list of tensors to be used for differentiation.
grad_ys: Optional. A `Tensor` or list of tensors the same size as
    `ys` and holding the gradients computed for each y in `ys`.
stop_gradients: Optional. A `Tensor` or list of tensors not to differentiate
    through.

In [ ]:
Returns:
  A list of `sum(dy/dx)` for each x in `xs`.

In [ ]:
tf.gradients(ys, xs)
注意，xs 中的每一个元素，必须要与 ys 相关，不相关的话，会报错
w1 = tf.Variable([[1,2]])
w2 = tf.Variable([[3,4]])
res = tf.matmul(w1, [[2],[1]])
grads = tf.gradients(res,[w1,w2])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    re = sess.run(grads)
    print(re)

In [ ]:
import tensorflow as tf
w1 = tf.get_variable('w1', shape=[3])
w2 = tf.get_variable('w2', shape=[3])
w3 = tf.get_variable('w3', shape=[3])
w4 = tf.get_variable('w4', shape=[3])

z1 = w1 + w2+ w3
z2 = w3 + w4

grads = tf.gradients([z1, z2], [w1, w2, w3, w4], 
                     grad_ys=[tf.convert_to_tensor([2.,2.,3.]), 
                              tf.convert_to_tensor([3.,2.,4.])])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(grads))

In [ ]:
tf.stop_gradient(input, name=None)

In [ ]:
Stops gradient computation.

When executed in a graph, this op outputs its input tensor as-is.

When building ops to compute gradients, this op prevents the contribution of
its inputs to be taken into account.  Normally, the gradient generator adds ops
to a graph to compute the derivatives of a specified 'loss' by recursively
finding out inputs that contributed to its computation.  If you insert this op
in the graph it inputs are masked from the gradient generator.  They are not
taken into account for computing gradients.

This is useful any time you want to compute a value with TensorFlow but need
to pretend that the value was a constant. Some examples include:

*  The *EM* algorithm where the *M-step* should not involve backpropagation
   through the output of the *E-step*.
*  Contrastive divergence training of Boltzmann machines where, when
   differentiating the energy function, the training must not backpropagate
   through the graph that generated the samples from the model.
*  Adversarial training, where no backprop should happen through the adversarial
   example generation process.

Args:
  input: A `Tensor`.
  name: A name for the operation (optional).

In [ ]:
阻挡节点 Backpropagation 的梯度

In [2]:
import tensorflow as tf
w1 = tf.Variable(2.0)
w2 = tf.Variable(2.0)
a = tf.multiply(w1, 3.0)
a_stoped = tf.stop_gradient(a)

# b=w1*3.0*w2
b = tf.multiply(a_stoped, w2)
gradients = tf.gradients(b, xs=[w1, w2])
print(gradients)

[None, <tf.Tensor 'gradients/Mul_1_grad/Mul_1:0' shape=() dtype=float32>]


In [ ]:
一个节点被stop之后，这个节点上的梯度，就无法再向前BP了
由于w1变量的梯度只能来自a节点，所以，计算梯度返回是None

In [ ]:
a = tf.Variable(1.0)
b = tf.Variable(1.0)
c = tf.add(a, b)
c_stoped = tf.stop_gradient(c)
d = tf.add(a, b)
e = tf.add(c_stoped, d)
gradients = tf.gradients(e, xs=[a, b])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(gradients)) #输出 [1.0, 1.0]

In [ ]:
虽然c节点被stop了，但是a,b还有从d传回的梯度，所以还是可以输出梯度值的

In [ ]:
# 高阶导数

In [6]:
with tf.device('/cpu:0'):
    a = tf.constant(1.)
    b = tf.pow(a, 2)
    grad = tf.gradients(ys=b, xs=a) # 一阶导
    print(grad[0])
    grad_2 = tf.gradients(ys=grad[0], xs=a) # 二阶导
    print(grad_2)
    grad_3 = tf.gradients(ys=grad_2[0], xs=a) # 三阶导
    print(grad_3)
    
with tf.Session() as sess:
    print(sess.run(grad_3))
    print(sess.run(grad_2))

Tensor("gradients_10/Pow_3_grad/Reshape:0", shape=(), dtype=float32, device=/device:CPU:0)
[<tf.Tensor 'gradients_11/gradients_10/Pow_3_grad/Pow_grad/Reshape:0' shape=() dtype=float32>]
[<tf.Tensor 'gradients_12/gradients_11/gradients_10/Pow_3_grad/Pow_grad/Pow_grad/Reshape:0' shape=() dtype=float32>]
[0.0]
[2.0]


In [ ]:
tf.square

In [ ]:
tf.split

In [ ]:
tf.matmul
將a,b兩個矩陣相乘，如果需要事先轉置的話，可以把個別的選項調成True，如果矩陣裏面包括很多0的話，可以調用spare=True轉為更有效率的演算法

In [ ]:
tf.mul
實現兩個矩陣點乘，兩個矩陣必須要相同維度

In [ ]:
tf.one_hot

In [ ]:
tf.Variable.eval(session=None)

函數解說：顯示出某個tensor變數的值

In [ ]:
tf.Variable.assign(value, use_locking=False)

函數解說：將tensor中的變數用value值取代，這function可以用來實現double DQN

In [ ]:
tf.Variable.assign_sub(delta, use_locking=False)

函數解說：將tensor中的變數-delta值，然後取代